## Imports

In [1]:
from pymatgen.transformations.standard_transformations import AutoOxiStateDecorationTransformation
import time
import numpy as np
from pymatgen.core.periodic_table import Species
import pandas as pd
from pymatgen.core.structure import Structure

## Class to add oxidation states to structure

In [2]:
aosdt = AutoOxiStateDecorationTransformation()

## Get testing structure and add oxidation states

In [3]:
structure = Structure.from_file('mp-4681.vasp')
oxi_structure = aosdt.apply_transformation(structure)

In [4]:
species = np.unique([s.specie for s in oxi_structure])
oxi_dict = {}
for s in species:
    oxi_dict[str(s.element)] = s.oxi_state

## Compute Matminer GII - Benchmarking

In [5]:
from matminer.featurizers.structure.bonding import GlobalInstabilityIndex
# matminer GlobalInstabilityIndex uses bvparm16.cif, which is the default for GIICalculator

In [6]:
start_time = time.time()
mm_gii_calc = GlobalInstabilityIndex(r_cut=4)
mm_gii = mm_gii_calc.featurize(oxi_structure)[0]
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.4426238536834717 seconds ---


In [7]:
mm_gii

0.2295876515555516

## Compute GIICalculator GII

In [8]:
from gii_minimization import GIICalculator

In [9]:
start_time = time.time()
#gii_calc = GIICalculator(method='CrystalNN')
gii_calc = GIICalculator(method='Cutoff', cutoff=4)
gii = gii_calc.GII(oxi_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.10015511512756348 seconds ---


In [10]:
gii

0.22958765155555164

In [11]:
gii_calc.params_dict

{'Cation': [Species Na+, Species Nb5+],
 'Anion': [Species O2-, Species O2-],
 'R0': [1.803, 1.911],
 'B': [0.37, 0.37]}

## Perform the Site Optimization

In [12]:
from gii_minimization import SiteClusterOptimization

In [35]:
sco = SiteClusterOptimization()

In [36]:
start_time = time.time()
final_structure = sco.minimize_cluster_di_squared(oxi_structure, 16) # pass the index of site to be optimized
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 1.5121281147003174 seconds ---


In [37]:
start_time = time.time()
start_gii = gii_calc.GII(oxi_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.07842350006103516 seconds ---


In [38]:
start_gii

0.22958765155555164

In [39]:
start_time = time.time()
final_gii = gii_calc.GII(final_structure, use_sym=False)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.17808842658996582 seconds ---


In [40]:
final_gii

0.22521258089793017

## Perform the Structure Optimization 

In [ ]:
from gii_minimization import GIIMinimizer

In [ ]:
gii_minimizer = GIIMinimizer(oxi_structure, 0.001)

In [ ]:
optimized_structure = gii_minimizer.gii_minimization(opt_method='max')